# Model Selection 

- $r^2$
- Adjusted $r^2$
- AIC
- BIC

## Linear Regression Example

Which model is better if number of parameters differ?

In [ ]:
#!pip install sympy

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### Coefficient of Determination

- Regression Sum of Squares $SSR = \sum(\hat{y_i} - \bar{y})^2$

- Error Sum of Squares $SSE = \sum(y_i - \hat{y_i})^2$

- Total Sum of Squares $SSTO = \sum(y_i - \bar{y})^2$

$$r^2 = \frac{SSR}{SSTO} = 1 - \frac{SSE}{SSTO}$$

In [ ]:
boston = load_boston()
X = boston.data
y = boston.target
bdf = pd.DataFrame(X, columns=boston.feature_names)
bdf['target'] = y

In [ ]:
#the boston data
bdf.head()

In [ ]:
#split, fit, score
def lin_reg_fitter(X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    lr = LinearRegression()

    lr.fit(X_train, y_train)
    
    return lr.score(X_train, y_train)  

In [ ]:
#create lists of scores
cols = []
scores = []
for col in boston.feature_names:
    cols.append(col)
    score = lin_reg_fitter(bdf[cols], bdf['target'])
    scores.append(score)

In [ ]:
pd.DataFrame({'num_features': [i for i in range(len(boston.feature_names))], 'r2': scores})

### Adjusted $r^2$

Adjust the $SSE$ and $SSTO$ by degrees of freedom.

$$r^2_a = 1 - \frac{SSE/(n-p)}{SSTO/(n-1)} = 1 - \frac{MSE}{MSTO}$$

$$r^2_a = r^2 - ( \frac{p-1}{n-p}) (1 - r^2)$$

### Generic Information Criteria

$$xIC = -2ln(L) + complexity ~term$$

$L$ is maximized likelihood.

### Akaike Information Criterion

$$AIC = -2 ln(L) + 2p$$

$$AIC = n \ln(SSE_p/n) + 2p$$

### Bayesian Information Criterion

$$BIC = -2 ln(L) + p ln(n)$$

$$BIC = n\ln(SSE_p/n) + p \ln(n)$$

Penalizes more complex more ($n \geq 8$).

### Comparison

$$r^2_a \quad \text{or}\quad  AIC \quad \text{or} \quad BIC$$

## Statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
lr = sm.OLS(boston.target, bdf['CHAS'] + bdf['INDUS'])

In [ ]:
results = lr.fit()

In [ ]:
print(lr.fit().summary())

In [ ]:
results.rsquared_adj

In [ ]:
results.rsquared

In [ ]:
results.aic

In [ ]:
results.bic

### Logistic Regression

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
iris = load_iris()

In [ ]:
X = iris.data
y = iris.target

In [ ]:
ss = MaxAbsScaler()
X = ss.fit_transform(X)
y = ss.fit_transform(y.reshape(-1,1))

In [ ]:
iris_df = pd.DataFrame(X, columns = iris.feature_names)

In [ ]:
model = sm.Logit(y, iris_df['sepal length (cm)'])

In [ ]:
model.fit()

In [ ]:
model.fit().summary2()

In [ ]:
model = sm.Logit( y, iris_df['sepal length (cm)'] + iris_df['petal length (cm)'])

In [ ]:
model.fit()

In [ ]:
model.fit().summary2()

### KMeans Clustering

In [ ]:
#https://stats.stackexchange.com/questions/90769/using-bic-to-estimate-the-number-of-k-in-kmeans

from sklearn import cluster
from scipy.spatial import distance
import sklearn.datasets
from sklearn.preprocessing import StandardScaler
import numpy as np

def compute_bic(kmeans,X):
    """
    Computes the BIC metric for a given clusters

    Parameters:
    -----------------------------------------
    kmeans:  List of clustering object from scikit learn

    X     :  multidimension np array of data points

    Returns:
    -----------------------------------------
    BIC value
    """
    # assign centers and labels
    centers = [kmeans.cluster_centers_]
    labels  = kmeans.labels_
    #number of clusters
    m = kmeans.n_clusters
    # size of the clusters
    n = np.bincount(labels)
    #size of data set
    N, d = X.shape

    #compute variance for all clusters beforehand
    cl_var = (1.0 / (N - m) / d) * sum([sum(distance.cdist(X[np.where(labels == i)], [centers[0][i]], 
             'euclidean')**2) for i in range(m)])

    const_term = 0.5 * m * np.log(N) * (d+1)

    BIC = np.sum([n[i] * np.log(n[i]) -
               n[i] * np.log(N) -
             ((n[i] * d) / 2) * np.log(2*np.pi*cl_var) -
             ((n[i] - 1) * d/ 2) for i in range(m)]) - const_term

    return(BIC)

In [ ]:
# IRIS DATA
iris = sklearn.datasets.load_iris()
X = iris.data[:, :4]  # extract only the features
#Xs = StandardScaler().fit_transform(X)
Y = iris.target

ks = range(1,10)

# run 9 times kmeans and save each result in the KMeans object
KMeans = [cluster.KMeans(n_clusters = i, init="k-means++").fit(X) for i in ks]

# now run for each cluster the BIC computation
BIC = [compute_bic(kmeansi,X) for kmeansi in KMeans]

plt.bar(ks, BIC)

### Gaussian Mixture Models

In [ ]:
import numpy as np
import itertools

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture
# Number of samples per component
n_samples = 500

# Generate random sample, two components
np.random.seed(0)
C = np.array([[0., -0.1], [1.7, .4]])
X = np.r_[np.dot(np.random.randn(n_samples, 2), C),
          .7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]

In [ ]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,
                                      covariance_type=cv_type)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                              'darkorange'])
clf = best_gmm
bars = []

In [ ]:
# Plot the BIC scores
plt.figure(figsize=(8, 6))
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

In [ ]:
# Plot the winner
splot = plt.subplot(2, 1, 2)
Y_ = clf.predict(X)
for i, (mean, cov, color) in enumerate(zip(clf.means_, clf.covariances_,
                                           color_iter)):
    v, w = linalg.eigh(cov)
    if not np.any(Y_ == i):
        continue
    plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180. * angle / np.pi  # convert to degrees
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)

plt.xticks(())
plt.yticks(())
plt.title('Selected GMM: full model, 2 components')
plt.subplots_adjust(hspace=.35, bottom=.02)
plt.show()